In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!sudo fuser -v /dev/nvidia* -k

In [ ]:
!pip install gradio
!pip install huggingface_hub
import huggingface_hub
huggingface_hub.login(token="SECRET")
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7



Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import gradio as gr

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    GenerationConfig
)
from peft import LoraConfig, PeftModel

# from google.colab import drive
# drive.mount('/content/drive')



In [ ]:
model_name_or_path = "meta-llama/Llama-2-13b-hf"
adapter_path = "bgilles/PsychometricLLaMA"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
# Fixing some of the early LLaMA HF conversion issues.
tokenizer.bos_token_id = 1

# Load the model (use f16 for faster inference)

model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    torch_dtype=torch.float16,
    device_map={"": 0},
    load_in_4bit=True,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4',
    )
)

model = PeftModel.from_pretrained(model, adapter_path)
model.eval()





/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 5120, padding_idx=0)
        (layers): ModuleList(
          (0-39): 40 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=5120, out_features=5120, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=5120, out_features=5120, bias=False

In [ ]:
tokenizer_with_prefix_space = AutoTokenizer.from_pretrained(model_name_or_path, add_prefix_space=True)

def get_tokens_as_list(word_list):
    "Converts a sequence of words into a list of tokens"
    tokens_list = []
    for word in word_list:
        tokenized_word = tokenizer_with_prefix_space([word], add_special_tokens=False).input_ids[0]
        tokens_list.append(tokenized_word)
    return tokens_list

bad_words = get_tokens_as_list(["#","##","###","/n","\n###","###\n"])
bad_words.append([13])

def generate_items_sample_p(prompt, max_new_tokens=200, temperature=0.9, num_return_sequences=15, num_batches=1, top_p=0.90):



    config = GenerationConfig(
        max_new_tokens=max_new_tokens,  # Maximum length of the generated text
        do_sample=True,  # Whether or not to use sampling in generation
        temperature=temperature,  # The value used to module the next token probabilities,
        num_return_sequences=num_return_sequences,
        remove_invalid_values=True,
        top_p = top_p,
        bad_words_ids =bad_words
    )

    input = tokenizer(prompt, return_tensors="pt")
    input.to('cuda')

    out_dec = []
    for _ in range(num_batches):
        with torch.no_grad():
            out = model.generate(**input, generation_config=config)
            batch_out_dec = tokenizer.batch_decode(out, skip_special_tokens=True)

        for i in range(len(batch_out_dec)):
            batch_out_dec[i] = batch_out_dec[i].replace(prompt, "")

        out_dec.extend(list(dict.fromkeys(batch_out_dec)))

        del out
        torch.cuda.empty_cache()

    del input
    torch.cuda.empty_cache()

    return out_dec

Predict outputs for the test-set.


In [ ]:
def format_prompt(construct="NA", definition="NA", subconstruct="NA", subdefinition="NA", difficulty="NA",inverted="FALSE"):
    return (
        f"###Construct: "
        f"{construct} \n"
        f" ###C_Definition: "
        f"{definition} \n"
        f" ###Subconstruct: "
        f"{subconstruct} \n"
        f" ###S_Definition: "
        f"{subdefinition} \n"
        f" ###Difficulty: "
        f"{difficulty} \n"
        f" ###Inverted: "
        f"{inverted} \n"
        f" ###Item:"
    )

In [ ]:
na_prompt = format_prompt()

na_og_prompt = """###Construct: NA
###C_definition: NA
###Subconstruct: NA
###S_defintion: NA
###Inverted: FALSE
###Item:"""




'###Construct: NA\n###C_definition: NA\n###Subconstruct: NA\n###S_defintion: NA\n###Inverted: FALSE\n###Item:'

In [ ]:
generate_text()

NameError: name 'generate_text' is not defined

In [ ]:
def generate_text(construct="Extraversion", definition="NA", subconstruct="NA", subdefinition="NA", difficulty="NA",inverted="FALSE",temperature=0.9,n_samples=10,batches=2):
    prompt = format_prompt(construct, definition, subconstruct, subdefinition,inverted, difficulty)
    tmp = generate_items_sample_p(prompt,temperature=temperature, max_new_tokens=30,num_return_sequences=n_samples,num_batches=batches)
    return tmp


demo = gr.Interface(fn=generate_text,
                    title="Psychometric LLaMA - detailed transformer based item generation",
                    description = "To use this generator to create likert-items, fill in construct names and definitions. You can also leave parts blank to try the default generations. Examples are provided below. \n For more or less content variance try to change the temperature parameter. Best accuracy with limited creativity is archived at t = .7 - .9. \n The number of generated samples is n_samples * number of batches." ,
                    inputs=["text","text","text","text",gr.Slider(minimum=1,maximum=5,step=1,value=3),"text",gr.Slider(minimum=0,maximum=2,step=0.1,value=0.9),gr.Slider(minimum=1,maximum=15,step=1,value=10),gr.Slider(minimum=1,maximum=3,step=1,value=2)],
                    outputs="text",
                    examples=[
                        ["Emotional intelligence","Trait Emotional intelligence refers to self-reported emotion-related dispositions",
                         "emotional regulation of the self","NA","3","FALSE"],
                        [
                            "Conscientiousness","Conscientiousness describes socially prescribed impulse control that facilitates task- and goal-directed behavior, such as thinking before acting, delaying gratification, following normas and rules, and planning, organizing, and prioritizing tasks.",
                            "NA","NA","3","FALSE"
                        ],
                        [
                            "Germanness","This construct measures how German somebody is. It includes accents, food choices, hobbys like golfing etc, and tendencys like always being punctual.",
                            "NA","NA","3","FALSE"
                        ],
                        [
                            "Trust in AI","Trust in AI describes the individuel trust somebody has into AI applications at the workspace.",
                            "Trusting Data","Trusting data descibes the degree to which people trust into the used training data in a variety of contexts.","3","FALSE"
                        ]
                    ])

demo.launch(share=False,debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
